In [1]:
import scipy.stats as ss
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
to_fit = pd.read_csv("Data/costs_input_new.csv")

In [3]:
to_fit.head()

,Variable Name,Value,Lower Bound,Upper Bound
0,cLevel1NurseryAdmissionPerDay,703.0,234.00,1172.00
1,cLevel1NursingCarePerDay,410.9,273.93,684.83
2,cLevel2NICUAdmissionPerDay,3066.0,1022.00,5109.00
3,cLevel2NursingCarePerDay,501.1,334.07,835.17
4,cGentamicin1,21.0,7.00,34.00


In [4]:
# fits an encompassing gamma distribution to an expected value of value,
# 2.5th percentile of lower_bound, and 97.5th percentile of upper_bound
# scale_lower_bound and scale_upper_bound are guesses as to what the
# scale should be.

def fit_encompassing_gamma_distr(value, lower_bound, upper_bound):

    scale_lower_bound = np.around((abs(lower_bound - value)/2)**2 / value, 2)
    scale_upper_bound = scale_lower_bound + 10
    linspace_points = 1001
    search_range = np.linspace(scale_lower_bound, scale_upper_bound, linspace_points)

    for scale in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
        ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            a = value / scale
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    scale_lower_bound = int((((upper_bound - value)/1.8)**2 / value) / 100) * 100
    scale_upper_bound = scale_lower_bound + 100000
    linspace_points = 1001
    search_range = np.linspace(scale_lower_bound, scale_upper_bound, linspace_points)

    for scale in search_range:
        ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
        ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)
        
        
        if ppf_975 > upper_bound and ppf_025 < lower_bound:
            a = value / scale
            print("a: %.2f \t scale: %.2f" % (a, scale))           
            return a, scale
        
    print("No match found")
    return 0, 0

In [5]:
to_fit["a"], to_fit["scale"] = np.vectorize(fit_encompassing_gamma_distr)(to_fit["Value"], to_fit["Lower Bound"], to_fit["Upper Bound"])

<ipython-input-4-956dc2bf0b6d>:29: RuntimeWarning: divide by zero encountered in double_scalars
  ppf_975 = ss.gamma.ppf(0.975, a=value/scale, scale=scale)
/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:2004: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
<ipython-input-4-956dc2bf0b6d>:30: RuntimeWarning: divide by zero encountered in double_scalars
  ppf_025 = ss.gamma.ppf(0.025, a=value/scale, scale=scale)


a: 3.52 	 scale: 200.00
a: 3.52 	 scale: 200.00
a: 4.11 	 scale: 100.00
a: 5.11 	 scale: 600.00
a: 5.01 	 scale: 100.00
a: 5.19 	 scale: 4.05
a: 5.19 	 scale: 11.57
a: 3.41 	 scale: 200.00
a: 1.72 	 scale: 100.00
a: 5.26 	 scale: 19.78
a: 86.11 	 scale: 62.46
a: 4.10 	 scale: 126900.00
a: 12.78 	 scale: 89000.00
a: 15.98 	 scale: 637800.00


In [6]:
to_fit["gamma_mean"] = to_fit["a"] * to_fit["scale"]
to_fit["gamma_2.5pct"] = ss.gamma.ppf(0.025, a=to_fit["a"], scale=to_fit["scale"])
to_fit["gamma_97.5pct"] = ss.gamma.ppf(0.975, a=to_fit["a"], scale=to_fit["scale"])

In [7]:
to_fit

,Variable Name,Value,Lower Bound,Upper Bound,a,scale,gamma_mean,gamma_2.5pct,gamma_97.5pct
0,cLevel1NurseryAdmissionPerDay,703.0,234.00,1172.00,3.515000,200.00,703.0,1.704060e+02,1.605898e+03
1,cLevel1NursingCarePerDay,410.9,273.93,684.83,4.109000,100.00,410.9,1.145400e+02,8.930806e+02
2,cLevel2NICUAdmissionPerDay,3066.0,1022.00,5109.00,5.110000,600.00,3066.0,1.011094e+03,6.240354e+03
3,cLevel2NursingCarePerDay,501.1,334.07,835.17,5.011000,100.00,501.1,1.629630e+02,1.025751e+03
4,cGentamicin1,21.0,7.00,34.00,5.185185,4.05,21.0,6.996807e+00,4.256126e+01
5,cAmpicillin1,60.0,20.00,100.00,5.185825,11.57,60.0,1.999260e+01,1.215992e+02
6,cInitialNICUTests,682.0,227.00,1137.00,3.410000,200.00,682.0,1.605436e+02,1.573471e+03
7,cBloodCulture1,172.0,57.00,287.00,1.720000,100.00,172.0,1.614264e+01,5.077167e+02
8,cCSF,104.0,35.00,174.00,5.257836,19.78,104.0,3.498789e+01,2.099335e+02
9,cReadmission,5378.5,4302.80,6454.19,86.111111,62.46,5378.5,4.302761e+03,6.572458e+03


In [8]:
to_fit.to_csv("Data/costs_output_new.csv")